In [175]:
# https://keras.io/examples/nlp/ner_transformers/

In [176]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from datasets import load_dataset
from collections import Counter
from conlleval import evaluate

In [177]:
conll_data = load_dataset("conll2003")

In [178]:
def export_to_file(export_file_path, data):
    with open(export_file_path, "w") as f:
        for record in data:
            ner_tags = record["ner_tags"]
            tokens = record["tokens"]
            if len(tokens) > 0:
                f.write(
                    str(len(tokens))
                    + "\t"
                    + "\t".join(tokens)
                    + "\t"
                    + "\t".join(map(str, ner_tags))
                    + "\n"
                )


os.mkdir("data")
export_to_file("./data/conll_train.txt", conll_data["train"])
export_to_file("./data/conll_val.txt", conll_data["validation"])

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data'

In [179]:
def make_tag_lookup_table():
    iob_labels = ["B", "I"]
    ner_labels = ["PER", "ORG", "LOC", "MISC"]
    all_labels = [(label1, label2) for label2 in ner_labels for label1 in iob_labels]
    all_labels = ["-".join([a, b]) for a, b in all_labels]
    all_labels = ["[PAD]", "O"] + all_labels
    return dict(zip(range(0, len(all_labels) + 1), all_labels))


mapping = make_tag_lookup_table()
print(mapping)

{0: '[PAD]', 1: 'O', 2: 'B-PER', 3: 'I-PER', 4: 'B-ORG', 5: 'I-ORG', 6: 'B-LOC', 7: 'I-LOC', 8: 'B-MISC', 9: 'I-MISC'}


In [180]:
all_tokens = sum(conll_data["train"]["tokens"], [])
all_tokens_array = np.array(list(map(str.lower, all_tokens)))

counter = Counter(all_tokens_array)
print(len(counter))

num_tags = len(mapping)
vocab_size = 20000

# We only take (vocab_size - 2) most commons words from the training data since
# the `StringLookup` class uses 2 additional tokens - one denoting an unknown
# token and another one denoting a masking token
vocabulary = [token for token, count in counter.most_common(vocab_size - 2)]

# The StringLook class will convert tokens to token IDs
lookup_layer = keras.layers.StringLookup(vocabulary=vocabulary)

21009


In [181]:
train_data = tf.data.TextLineDataset("./data/conll_train.txt")
val_data = tf.data.TextLineDataset("./data/conll_val.txt")

In [182]:
print(list(train_data.take(1).as_numpy_iterator()))

[b'9\tEU\trejects\tGerman\tcall\tto\tboycott\tBritish\tlamb\t.\t3\t0\t7\t0\t0\t0\t7\t0\t0']


In [183]:
def map_record_to_training_data(record):
    record = tf.strings.split(record, sep="\t")
    length = tf.strings.to_number(record[0], out_type=tf.int32)
    tokens = record[1 : length + 1]
    tags = record[length + 1 :]
    tags = tf.strings.to_number(tags, out_type=tf.int64)
    tags += 1
    return tokens, tags
    
def lowercase_and_convert_to_ids(tokens):
    tokens = tf.strings.lower(tokens)
    return lookup_layer(tokens)
# We use `padded_batch` here because each record in the dataset has a
# different length.

batch_size = 32
train_dataset = (
    train_data.map(map_record_to_training_data)
    .map(lambda x, y: (lowercase_and_convert_to_ids(x), y))
    .padded_batch(batch_size)
)
val_dataset = (
    val_data.map(map_record_to_training_data)
    .map(lambda x, y: (lowercase_and_convert_to_ids(x), y))
    .padded_batch(batch_size)
)

In [184]:
#train_iterator = iter(train_dataset)
first_batch = next(train_iterator)
tokens, tags = first_batch
tokens, tags

(<tf.Tensor: shape=(32, 16), dtype=int64, numpy=
 array([[  512,  8781,  8782,    10,     2,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [   18,  2447,    30,  2717,  8245,     9,  1098,  4729,   391,
             3,   512,  3046,   714,     3,  1486,  6347],
        [ 1042,    38,    67,    10,   194,  2229,     2,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [   87,   320,    17,  1077,     2,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [   18,  2230,    30,  2448,  5253,     9,  1324,  5914,  3452,
             3,  1324,  4730,   352,   452,     0,     0],
        [  899, 13808, 13809,     3,  1400, 13810, 13811,    10,     2,
             0,     0,     0,     0,     0,     0,     0],
        [ 1769, 13812,     2,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [   18,  1904,    30,  271

In [210]:
class CustomNonPaddingTokenLoss(keras.losses.Loss):
    def __init__(self, name="custom_ner_loss"):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        loss_fn = keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=keras.losses.Reduction.NONE
        )
        loss = loss_fn(y_true, y_pred)
        mask = tf.cast((y_true > 0), dtype=tf.float32)
        loss = loss * mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)
loss = CustomNonPaddingTokenLoss()

In [204]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=2, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                keras.layers.Dense(ff_dim, activation="relu"),
                keras.layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training=True):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [205]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
    def call(self, inputs):
        maxlen = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        position_embeddings = self.pos_emb(positions)
        token_embeddings = self.token_emb(inputs)
        return token_embeddings + position_embeddings

In [206]:
class NERModel(keras.Model):
    def __init__(
        self, num_tags, vocab_size, maxlen=128, embed_dim=32, num_heads=2, ff_dim=32
    ):
        super().__init__()
        self.embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
        self.dropout1 = layers.Dropout(0.1)
        self.ff = layers.Dense(ff_dim, activation="relu")
        self.dropout2 = layers.Dropout(0.1)
        self.ff_final = layers.Dense(num_tags, activation="softmax")

    def call(self, inputs, training=False):
        x = self.embedding_layer(inputs)
        x = self.transformer_block(x)
        x = self.dropout1(x, training=training)
        x = self.ff(x)
        x = self.dropout2(x, training=training)
        x = self.ff_final(x)
        return x

In [207]:
num_tags

10

In [211]:
ner_model = NERModel(num_tags, vocab_size, embed_dim=32, num_heads=4, ff_dim=64)
padding_token = 0
ner_model.compile(optimizer="adam", loss=loss)
ner_model.fit(train_dataset, epochs=10)

Epoch 1/10
439/439 [==============================] - 10s 19ms/step - loss: 0.6225
Epoch 2/10
439/439 [==============================] - 9s 21ms/step - loss: 0.2509
Epoch 3/10
439/439 [==============================] - 10s 22ms/step - loss: 0.1542
Epoch 4/10
439/439 [==============================] - 10s 22ms/step - loss: 0.1214
Epoch 5/10
439/439 [==============================] - 10s 23ms/step - loss: 0.0955
Epoch 6/10
439/439 [==============================] - 10s 24ms/step - loss: 0.0782
Epoch 7/10
439/439 [==============================] - 10s 22ms/step - loss: 0.0612
Epoch 8/10
439/439 [==============================] - 10s 22ms/step - loss: 0.0532
Epoch 9/10
439/439 [==============================] - 10s 22ms/step - loss: 0.0472
Epoch 10/10
439/439 [==============================] - 10s 22ms/step - loss: 0.0424


In [217]:
def tokenize_and_convert_to_ids(text):
    tokens = text.split()
    return lowercase_and_convert_to_ids(tokens)
    
# Sample inference using the trained model
sample_input = tokenize_and_convert_to_ids(
    "british lamb"
)
sample_input = tf.reshape(sample_input, shape=[1, -1])
print(sample_input)

output = ner_model.predict(sample_input)
prediction = np.argmax(output, axis=-1)[0]
prediction = [mapping[i] for i in prediction]

# eu -> B-ORG, german -> B-MISC, british -> B-MISC
print(prediction)

tf.Tensor([[ 215 5773]], shape=(1, 2), dtype=int64)
1/1 [==============================] - 1s 877ms/step
['B-ORG', 'O']


In [222]:
import pandas as pd
pd.DataFrame(sample_input)

,0,1
0,215,5773


In [223]:
output

array([[[1.1546319e-09, 1.5470059e-05, 8.0001610e-06, 1.0159526e-06,
         9.6417809e-01, 4.6157079e-06, 1.0047689e-03, 8.0988531e-09,
         3.4787692e-02, 2.8634406e-07],
        [8.4100918e-07, 7.2046286e-01, 1.2903955e-06, 3.5517319e-04,
         1.8976832e-06, 2.7638689e-01, 7.9034222e-04, 1.9615162e-03,
         3.0298065e-06, 3.6155077e-05]]], dtype=float32)

In [193]:
def calculate_metrics(dataset):
    all_true_tag_ids, all_predicted_tag_ids = [], []

    for x, y in dataset:
        output = ner_model.predict(x)
        predictions = np.argmax(output, axis=-1)
        predictions = np.reshape(predictions, [-1])

        true_tag_ids = np.reshape(y, [-1])

        mask = (true_tag_ids > 0) & (predictions > 0)
        true_tag_ids = true_tag_ids[mask]
        predicted_tag_ids = predictions[mask]

        all_true_tag_ids.append(true_tag_ids)
        all_predicted_tag_ids.append(predicted_tag_ids)

    all_true_tag_ids = np.concatenate(all_true_tag_ids)
    all_predicted_tag_ids = np.concatenate(all_predicted_tag_ids)

    predicted_tags = [mapping[tag] for tag in all_predicted_tag_ids]
    real_tags = [mapping[tag] for tag in all_true_tag_ids]
    
    #evaluate(real_tags, predicted_tags)
    return real_tags, predicted_tags

real_tags, predicted_tags = calculate_metrics(val_dataset)

1/1 [==============================] - 0s 35ms/step


In [194]:
#evaluate(real_tags[0], predicted_tags[0])